In [ ]:
from urllib.parse import urlencode
from urllib.parse import urljoin
from datetime import datetime
from requests_oauthlib import OAuth1
import requests
import copy
import json
import time
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait

import warnings
warnings.filterwarnings('ignore')

# from config import params

In [ ]:
params_to_be_set = {
    'YEAR': '2020',
    'S_MONTH': '06',
    'S_DATE': '08',
    'E_MONTH': '06',
    'E_DATE': '10',
}

In [ ]:
params = {
    'YEAR': params_to_be_set['YEAR'],
    'S_MONTH': params_to_be_set['S_MONTH'],
    'S_DATE': params_to_be_set['S_DATE'],
    'E_MONTH': params_to_be_set['E_MONTH'],
    'E_DATE': params_to_be_set['E_DATE'],
    
    # Secret key of Twitter API
    'CONSUMER_KEY': 'foLsfclN3YXISSrstIqdzGmri',
    'CONSUMER_SECRET_KEY': 'D5EoS9E6eQRUdhHBQSCv6rVgutu9EhunnCww6iNJfoyrtvPYMG',
    'ACCESS_TOKEN': '4096685054-24InfHUvJzq14IYz8g7wCXmVaQwLDxzyzYDtKQ1',
    'ACCESS_TOKEN_SECRET': 'V3Q1q5MTStmHBUTiHYBFftEBXLXxWe7LnaJnUlLZ5eVPH',
    
    # Request parameters for Twitter API
    'ROOT_URL': 'https://api.twitter.com/1.1/search/tweets.json?',
    'LANG': ['ja', 'en'],
    'TWEET_TYPE': 'extended',
    'COUNT': 100,  # the number of response per one request (max 100, default is 15)
    'RANGE': 10,  # the number of request (max 180/15minutes)
    'RESULT_TYPE': 'mixed',  # popular or mixed or recent
    'WORDS':  ['\"データサイエンス\"', '\"機械学習\"', '\"data science\"', '\"machine learning\"'],
    'SINCE': '{}-{}-{}'.format(params_to_be_set['YEAR'], params_to_be_set['S_MONTH'], params_to_be_set['S_DATE']),
    'UNTIL': '{}-{}-{}'.format(params_to_be_set['YEAR'], params_to_be_set['E_MONTH'], params_to_be_set['E_DATE']),
    
    # parameters for Wordpress
    'WP_URL': 'https://hophead-ds.com',
    'WP_USERNAME': 'user',
    'WP_PASSWORD': 'oAVL oteM IlaB cTdi gmF6 5MMP',
    'STATUS': 'draft',
    'CATEGORY': [5],
    'SLUG': '{}-{}-{}-twitter-auto-post'.format(
        params_to_be_set['YEAR'], params_to_be_set['E_MONTH'], params_to_be_set['E_DATE']
    ),
    'TITLE': 'データサイエンス・機械学習 Twiiter 投稿まとめ {}/{}/{}-{}/{}/{}'.format(
        params_to_be_set['YEAR'], params_to_be_set['S_MONTH'], params_to_be_set['S_DATE'], 
        params_to_be_set['YEAR'], params_to_be_set['E_MONTH'], params_to_be_set['E_DATE']
    ),
    'TAG_IDS': [],
    'MEDIA_ID': None,
    
    # parameters for selenium with chromedriver
    'DRIVER_DIR': '../config/chromedriver_v83',
    'DEEPL_URL': 'https://www.deepl.com/ja/translator',
    'PAGE_LOAD_TIMEOUT': 10,
}

## Search popular tweets I'm interested in.

- API reference: http://westplain.sakuraweb.com/translate/twitter/Documentation/REST-APIs/Public-API/GET-search-tweets.cgi

In [ ]:
def search_tweets(params, debug=False):
    tweets = []
    auth = OAuth1(
        params['CONSUMER_KEY'],
        params['CONSUMER_SECRET_KEY'],
        params['ACCESS_TOKEN'],
        params['ACCESS_TOKEN_SECRET']
    )
    
    for lang in params['LANG']:
        for word in params['WORDS']:
            cnt = 0
            print(lang, word)
            word += ' exclude:retweets exclude:replies'

            while True:
                if cnt > params['RANGE']:
                    break

                if cnt == 0:
                    url_args = urlencode({
                        'lang': lang,
                        'q': word,
                        'count': str(params['COUNT']),
                        'result_type': params['RESULT_TYPE'],
                        'tweet_mode':params['TWEET_TYPE'],
                        'since': params['SINCE'],
                        'until': params['UNTIL'],
                    })
                    response = requests.get(url=params['ROOT_URL'] + url_args, auth=auth)
                    data = response.json()['statuses']
                    if len(data) == 0:
                        break
                else:
                    for tweet in data:
                        max_id = int(tweet['id']) - 1
                        if tweet in tweets:
                            continue
                        tweets.append(tweet)
                        if debug:
                            print('For loop: {}, tweet created at: {}'.format(cnt, tweet['created_at']))
                    url_args = urlencode({
                        'lang': lang,
                        'q': word,
                        'count': str(params['COUNT']),
                        'result_type': params['RESULT_TYPE'],
                        'tweet_mode':params['TWEET_TYPE'],
                        'since': params['SINCE'],
                        'until': params['UNTIL'],
                        'max_id': str(max_id),
                    })
                    response = requests.get(url=params['ROOT_URL'] + url_args, auth=auth)
                    try:
                        data = response.json()['statuses']
                        if len(data)==0:
                            break
                    except KeyError:  # if the number of requests is over the limit.
                        print('Error: the number of requests is over the limit.')
                        break

                cnt += 1

    return tweets


def prune_query_result(tweet):
    tweet_pruned = copy.copy(tweet)
    prune_keys = [
        'id_str',
        'truncated',
        'entities',
        'extended_entities',
        'metadata',
        'source',
        'in_reply_to_status_id',
        'in_reply_to_status_id_str',
        'in_reply_to_user_id',
        'in_reply_to_user_id_str',
        'in_reply_to_screen_name',
        'geo',
        'coordinates',
        'place',
        'contributors',
        'is_quote_status',
        'favorited', 
        'retweeted', 
        'possibly_sensitive', 
        'quoted_status',
        'quoted_status_id',
        'quoted_status_id_str',
        'user',
    ]
    tweet_pruned['screen_name'] = tweet_pruned['user']['screen_name']
    for key in prune_keys:
        try:
            del tweet_pruned[key]
        except KeyError:
            pass
    return tweet_pruned


def extract_popular_tweets(tweets, threshold=100):
    popular_tweets = []
    popular_tweets_ids = []
    for tweet in tweets:
        if tweet['favorite_count']>=threshold:
            if tweet['id'] not in popular_tweets_ids:
                popular_tweets_ids.append(tweet['id'])
                tweet_pruned = prune_query_result(tweet)
                popular_tweets.append(tweet_pruned)
    popular_tweets.sort(key=lambda x: x['favorite_count'], reverse=True)
    
    return popular_tweets

In [ ]:
tweets = search_tweets(params, debug=True)

In [ ]:
len(tweets)

In [ ]:
popular_tweets = extract_popular_tweets(tweets, threshold=200)

In [ ]:
len(popular_tweets)

In [ ]:
import pandas as pd
pd.io.json.json_normalize(popular_tweets)

## Translate tweets in English

In [ ]:
def initialize_chrome_driver(params):
    chrome_options = Options()
    chrome_options.add_argument('--dns-prefetch-disable')
    chrome_options.add_argument('--disable-gpu')
    driver = Chrome(executable_path=params['DRIVER_DIR'], chrome_options=chrome_options)
    driver.set_page_load_timeout(params['PAGE_LOAD_TIMEOUT'])
    driver.maximize_window()
    return driver


def delete_emoji(text):
    return text.encode('ascii', 'ignore').decode('ascii')


def modify_tweet_text(text):
    text = delete_emoji(text)
    return text.replace('\n', ' ')


def translate_text_with_deepl(params, text, driver):
    driver.get(params['DEEPL_URL'])
    
    input_text_xpath = '//*[@id="dl_translator"]/div[1]/div[3]/div[2]/div/textarea'
    output_text_xpath = '//*[@id="dl_translator"]/div[1]/div[4]/div[3]/div[1]/textarea'
    language_select_xpath = '//*[@id="dl_translator"]/div[1]/div[4]/div[1]/div[1]/div[1]/button/div'
    target_language_xpath = '//*[@id="dl_translator"]/div[1]/div[4]/div[1]/div[1]/div[1]/div/button[1]'
    
    input_box = WebDriverWait(driver, 20).until(
        lambda driver: driver.find_element_by_xpath(input_text_xpath))
    output_box = WebDriverWait(driver, 20).until(
        lambda driver: driver.find_element_by_xpath(output_text_xpath))
    driver.execute_script('arguments[0].value = arguments[1]', input_box, text)
    
    select = WebDriverWait(driver, 20).until(
        lambda driver: driver.find_element_by_xpath(language_select_xpath))
    select.click()
    time.sleep(0.5)

    lang = WebDriverWait(driver, 20).until(
            lambda driver: driver.find_element_by_xpath(target_language_xpath))
    lang.click()
    time.sleep(5)

    translated_text = output_box.get_attribute('value')
    
    return translated_text


def translate_tweets(params, tweets, debug=False):
    tweet_translated = copy.copy(tweets)
    driver = initialize_chrome_driver(params)
    for tweet in tweet_translated:
        tweet_id = tweet['id']
        screen_name = tweet['screen_name']
        language = tweet['lang']
        full_text = modify_tweet_text(tweet['full_text'])
        
        if language == 'ja':
            translated_text = ''
        elif language == 'en':
            translated_text = translate_text_with_deepl(params, full_text, driver)
        tweet['translated_full_text'] = translated_text
        
        if debug:
            print('lang: {}, translated text:{} '.format(language, translated_text))
            
    driver.quit()
    return tweet_translated

In [ ]:
# popular_tweets[0:5]

In [ ]:
tweet_translated = translate_tweets(params, popular_tweets[0:5], debug=True)

## Post infomation about tweets on Wordpress

In [ ]:
def read_text_file(path):
    f = open(path)
    data = f.read() 
    f.close()
    return data


def make_html_for_tweet_embeded(popular_tweets):
    html_format_for_tweet_ja = read_text_file('../text/html_format_for_tweet_ja.txt')
    html_format_for_tweet_en = read_text_file('../text/html_format_for_tweet_en.txt')
    html_for_tweets_ja = ''
    html_for_tweets_en = ''
    for tweet in popular_tweets:
        tweet_id = tweet['id']
        screen_name = tweet['screen_name']
        language = tweet['lang']
        translated_full_text = tweet['translated_full_text']
        if language == 'ja':
            html_to_be_added = html_format_for_tweet_ja.format(
                    SCREEN_NAME=screen_name,
                    ID=tweet_id
            ) + '\n'
            html_for_tweets_ja += html_to_be_added
        elif language == 'en':
            html_to_be_added = html_format_for_tweet_en.format(
                    SCREEN_NAME=screen_name,
                    ID=tweet_id,
                    TRANSLATED_TEXT=translated_full_text
            ) + '\n'
            html_for_tweets_en += html_to_be_added
    return {
        'ja': html_for_tweets_ja,
        'en': html_for_tweets_en
    }


def make_content_as_html_for_wp(params, popular_tweets):
    html_for_tweets = make_html_for_tweet_embeded(popular_tweets)
    body_for_wp_post = read_text_file('../text/body_for_wp_post.txt')
    body_for_wp_post = body_for_wp_post.format(
        YEAR=params['YEAR'], 
        S_MONTH=params['S_MONTH'],
        S_DATE=params['S_DATE'],
        E_MONTH=params['E_MONTH'],
        E_DATE=params['E_DATE'],
        TWEETS_JA=html_for_tweets['ja'],
        TWEETS_EN=html_for_tweets['en'],
        
    )
    return body_for_wp_post


def post_article(params, tweets, debug=False):
    user_name = params['WP_USERNAME']
    pass_word = params['WP_PASSWORD']
    payload = {
        "status": params['STATUS'],
        "slug": params['SLUG'],
        "title": params['TITLE'],
        "content": make_content_as_html_for_wp(params, tweets),
        "date": datetime.now().isoformat(),
        "categories": params['CATEGORY'],
    }
    if params['MEDIA_ID'] is not None:
        payload['featured_media'] = params['MEDIA_ID']
    res = requests.post(
        urljoin(params['WP_URL'], "wp-json/wp/v2/posts"),
        data=json.dumps(payload),
        headers={'Content-type': "application/json"},
        auth=(user_name, pass_word)
    )
    if debug:
        print('----------\nTitle: 「{}」の投稿リクエスト結果:{}'.format(params['TITLE'], repr(res.status_code)))
    return res

In [ ]:
post_article(params, tweet_translated, debug=True)

In [ ]:
# requests.get(urljoin(params['WP_URL'], "wp-json/wp/v2/categories")).json()